# Scenario 2: A cross-functional team with one data scientist working on an ML model

MLflow setup:
- tracking server; yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.
To run this example we need to launch the mlflow server locally by running the following command in our terminal:<br/>
<mark >mlflow server --backend-store-uri sqlite:///backend.db</mark>  


In [4]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [5]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score


with mlflow.start_run():
    
    X, y = load_iris(return_X_y = True)
    
    params = {"C":0.1, "random_state":42}
    mlflow.log_params(params)
    
    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))
    
    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/home/iiitd/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: './artifacts_local/0/dfc1780e3d26470c95ee89b211f39300/artifacts'


In [7]:
mlflow.list_experiments()

[<Experiment: artifact_location='./artifacts_local/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>]

# Interacting with the model registry

In [11]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [12]:
client.list_registered_models()

[]

In [14]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id

#register a model
mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = "iris-classifier"
)

IndexError: list index out of range